In [ ]:
import pandas as pd
import numpy as np
import glob
import re
import os


In [ ]:
id = 'sl5tp300et9_20'
# Read all csv files
files = glob.glob(f'data/*{id}.csv')

df = pd.concat((pd.read_csv(file) for file in files))

# Clean up headers
df = df[pd.isna(df['Instrument']) == False]
df.reset_index(drop=True, inplace=True)


In [ ]:
# Calculate some columns
df['Credit/Debit'] = df['Quantity'] * df['Price'] * -1
date_regex = r"(\d{2}) (\w{3}) (\d{4})"
matches = df['Trigger Date'].apply(lambda row: re.search(date_regex, row))
df['Date'] = matches.apply(lambda match: match.group(1))
df['Month'] = matches.apply(lambda match: match.group(2))
df['Year'] = matches.apply(lambda match: match.group(3))
df['Datetime'] = df[['Year', 'Month', 'Date']].astype(str).apply('-'.join, 1)
df['Datetime'] = pd.to_datetime(df['Datetime'])
df = df.sort_values(by=['Datetime'])
df = df.drop_duplicates(subset=['Trigger Date', 'Instrument', 'Buy/Sell'])

# Save output
output_file = f'output_{id}.xlsx'
df.to_excel(output_file, sheet_name='Raw', index=False)


In [ ]:
# Analysis
df.pivot_table(index=['Month'], columns=['Year'], values=[
               'Credit/Debit'], aggfunc='sum', margins=True, sort=False).iloc[:, :-1]
